# 互评作业2: 频繁模式与关联规则挖掘
**姓名**：聂宇翔  
**学号**：1120161722  
**Github地址**：https://github.com/JerrryNie/dataMining

# 内容  
## 数据集：Wine Reviews
### 1. 处理数据集  
   - 处理数据集

### 2. 找出频繁模式
   - 寻找频繁模式并统计频繁模式信息
   - 可视化所找出的频繁模式

### 3. 导出关联规则并计算支持度和置信度
   - 定义关联规则
   - 挖掘关联规则
   - 计算支持度和置信度并对关联规则以及相应的支持度置信度进行可视化
   
### 4. 评价关联规则
   - Lift指标
   - Conviction指标
    
### 5. 可视化展示并对挖掘结果进行分析
   - 可视化展示
   - 对挖掘结果进行分析

### 1. 处理数据集
   - 处理数据集
   本部分将对该实验中所选用的Wine Reviews数据集进行导入、清理并可视化，为之后的频繁模式与关联规则挖掘做准备。

In [1]:
import pandas as pd
import numbers
import typing
import itertools
import collections

我们首先对数据进行加载，对该数据集中行数和列数进行统计，并简要可视化

In [2]:
dataPath = 'winemag-data-130k-v2.csv'
df = pd.read_csv(dataPath)
print('%d columns' % len(df.columns))
print('%d rows' % len(df.index))
df.head()

14 columns
129971 rows


,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


选择**country**，**designation**，**taster_name**，**province**，**variety**，**winery**这几列属性进行数据清理（将缺失值用nan进行填充），并在之后以这些属性为基础，进行关联规则挖掘。

In [3]:
columns = ['country','designation', 'province', 'taster_name', 'variety', 'winery']
df = df[columns].fillna('nan')
transactions = df.values.tolist()

print('%d transactions.' % len(transactions), end='\n\n')
for idx, t in enumerate(transactions[:5]):
    print('%d:' % idx, t, end='\n\n')
print('...')

129971 transactions.

0: ['Italy', 'Vulkà Bianco', 'Sicily & Sardinia', 'Kerin O’Keefe', 'White Blend', 'Nicosia']

1: ['Portugal', 'Avidagos', 'Douro', 'Roger Voss', 'Portuguese Red', 'Quinta dos Avidagos']

2: ['US', 'nan', 'Oregon', 'Paul Gregutt', 'Pinot Gris', 'Rainstorm']

3: ['US', 'Reserve Late Harvest', 'Michigan', 'Alexander Peartree', 'Riesling', 'St. Julian']

4: ['US', "Vintner's Reserve Wild Child Block", 'Oregon', 'Paul Gregutt', 'Pinot Noir', 'Sweet Cheeks']

...


### 2. 找出频繁模式

- 寻找频繁模式并统计频繁模式信息
为了在特定的transactions以及指定的支持度下进行频繁模式的查找，我们采用如下的函数设置，查找支持度为0.05的频繁模式。

In [4]:
def join(itemsets):
    i = 0
    while i < len(itemsets):
        skip = 1
        *first, last = itemsets[i]
        tail = [last]
        tail_append = tail.append
        for j in range(i + 1, len(itemsets)):
            *nFirst, nLast = itemsets[j]
            if first == nFirst:
                tail_append(nLast)
                skip += 1
            else:
                break
        first = tuple(first)
        for a, b in sorted(itertools.combinations(tail, 2)):
            yield first + (a,) + (b,)
        i += skip


def prune( itemsets, possible_itemsets):
    itemsets = set(itemsets)
    for possible_itemset in possible_itemsets:
        for i in range(len(possible_itemset) - 2):
            removed = possible_itemset[:i] + possible_itemset[i + 1 :]
            if removed not in itemsets:
                break
        else:
            yield possible_itemset


def apriori(itemsets):
    possible_extensions = join(itemsets)
    yield from prune(itemsets, possible_extensions)


def find(transactions,min_support,max_length = 8,verbosity = 0,):
    transaction_sets = [set(t) for t in transactions if len(t) > 0]
    transactions = transaction_sets
    use_transaction = collections.defaultdict(lambda: True)

    counts = collections.defaultdict(int)
    num_transactions = 0
    for transaction in transactions:
        num_transactions += 1  
        for item in transaction:
            counts[item] += 1  

    large_itemsets = [
        (i, c)
        for (i, c) in counts.items()
        if (c / num_transactions) >= min_support
    ]

    if large_itemsets:
        large_itemsets = {1: {(i,): c for (i, c) in sorted(large_itemsets)}}
    else:
        return dict(), num_transactions

    issubset = set.issubset  
    k = 2
    while large_itemsets[k - 1] and (max_length != 1):
        itemsets_list = list(large_itemsets[k - 1].keys())
        C_k = list(apriori(itemsets_list))
        C_k_sets = [set(itemset) for itemset in C_k]

        if not C_k:
            break

        candidate_itemset_counts = collections.defaultdict(int)
        for row, transaction in enumerate(transactions):
            if not use_transaction[row]:
                continue
            found_any = False
            for candidate, candidate_set in zip(C_k, C_k_sets):
                if issubset(candidate_set, transaction):
                    candidate_itemset_counts[candidate] += 1
                    found_any = True
            if not found_any:
                use_transaction[row] = False

        C_k = [
            (i, c)
            for (i, c) in candidate_itemset_counts.items()
            if (c / num_transactions) >= min_support
        ]
        if not C_k:
            break

        large_itemsets[k] = {i: c for (i, c) in sorted(C_k)}

        k += 1
        if k > max_length:
            break

    return large_itemsets, num_transactions

itemsets, num_trans = find(
    transactions=transactions,
    min_support=0.05,
    verbosity=0,
)

对所找到而频繁模式进行可视化显示

In [5]:
for k, itemset in itemsets.items():
    print('%s itemsets:' % k)
    for items in itemset:
        print(items)
    print()

1 itemsets:
('Bordeaux-style Red Blend',)
('Cabernet Sauvignon',)
('California',)
('Chardonnay',)
('France',)
('Italy',)
('Kerin O’Keefe',)
('Michael Schachner',)
('Paul Gregutt',)
('Pinot Noir',)
('Red Blend',)
('Roger Voss',)
('Spain',)
('US',)
('Virginie Boone',)
('Washington',)
('nan',)

2 itemsets:
('Cabernet Sauvignon', 'US')
('California', 'Pinot Noir')
('California', 'US')
('California', 'Virginie Boone')
('California', 'nan')
('Chardonnay', 'US')
('France', 'Roger Voss')
('France', 'nan')
('Italy', 'Kerin O’Keefe')
('Italy', 'nan')
('Michael Schachner', 'Spain')
('Paul Gregutt', 'US')
('Pinot Noir', 'US')
('Roger Voss', 'nan')
('US', 'Virginie Boone')
('US', 'Washington')
('US', 'nan')

3 itemsets:
('California', 'Pinot Noir', 'US')
('California', 'US', 'Virginie Boone')
('California', 'US', 'nan')
('France', 'Roger Voss', 'nan')



### 3. 导出关联规则并计算支持度和置信度

- 定义关联规则

In [6]:
class Rule(object):
    _decimals = 3

    def __init__(
        self,
        lhs: tuple,
        rhs: tuple,
        count_full: int = 0,
        count_lhs: int = 0,
        count_rhs: int = 0,
        num_transactions: int = 0,
    ):
        self.lhs = lhs  # antecedent
        self.rhs = rhs  # consequent
        self.count_full = count_full
        self.count_lhs = count_lhs
        self.count_rhs = count_rhs
        self.num_transactions = num_transactions

    @property
    def confidence(self):
        try:
            return self.count_full / self.count_lhs
        except ZeroDivisionError:
            return None
        except AttributeError:
            return None

    @property
    def support(self):
        try:
            return self.count_full / self.num_transactions
        except ZeroDivisionError:
            return None
        except AttributeError:
            return None

    @staticmethod
    def _pf(s):
        return "{" + ", ".join(str(k) for k in s) + "}"

    def __str__(self):
        return "{} -> {}".format(self._pf(self.lhs), self._pf(self.rhs))

    def __eq__(self, other):
        return (set(self.lhs) == set(other.lhs)) and (
            set(self.rhs) == set(other.rhs)
        )

    def __hash__(self):
        return hash(frozenset(self.lhs + self.rhs))

    def __len__(self):
        return len(self.lhs + self.rhs)
    
    
    
def generate_rules_apriori(
    itemsets: typing.Dict[int, typing.Dict[tuple, int]],
    min_confidence: float,
    num_transactions: int,
    verbosity: int = 0,
):
    def count(itemset):
        return itemsets[len(itemset)][itemset]

    for size in itemsets.keys():
        if size < 2:
            continue
        if verbosity > 0:
            print(" Generating rules of size {}.".format(size))

        for itemset in itemsets[size].keys():
            for removed in itertools.combinations(itemset, 1):
                lhs = set(itemset).difference(set(removed))
                lhs = tuple(sorted(list(lhs)))
                conf = count(itemset) / count(lhs)
                if conf >= min_confidence:
                    yield Rule(
                        lhs,
                        removed,
                        count(itemset),
                        count(lhs),
                        count(removed),
                        num_transactions,
                    )

            H_1 = list(itertools.combinations(itemset, 1))
            yield from _ap_genrules(
                itemset, H_1, itemsets, min_confidence, num_transactions
            )



def _ap_genrules(
    itemset: tuple,
    H_m: typing.List[tuple],
    itemsets: typing.Dict[int, typing.Dict[tuple, int]],
    min_conf: float,
    num_transactions: int,
):

    def count(itemset):
        return itemsets[len(itemset)][itemset]

    if len(itemset) <= (len(H_m[0]) + 1):
        return

    H_m = list(apriori(H_m))
    H_m_copy = H_m.copy()

    for h_m in H_m:
        lhs = tuple(sorted(list(set(itemset).difference(set(h_m)))))
        if (count(itemset) / count(lhs)) >= min_conf:
            yield Rule(
                lhs,
                h_m,
                count(itemset),
                count(lhs),
                count(h_m),
                num_transactions,
            )
        else:
            H_m_copy.remove(h_m)
    if H_m_copy:
        yield from _ap_genrules(
            itemset, H_m_copy, itemsets, min_conf, num_transactions
        )

- 挖掘关联规则

In [7]:
rules = generate_rules_apriori(
    itemsets=itemsets, 
    min_confidence=0.1, 
    num_transactions=num_trans, 
    verbosity=0,
)
rules = list(rules)

- 计算支持度和置信度并对关联规则以及相应的支持度置信度进行可视化

In [8]:
rules_t = []
for rule in rules:
    valid = True
    for val in rule.lhs + rule.rhs:
        if 'nan' in val:
            valid = False
            break
    if valid:
        rules_t.append(rule)
rules = rules_t

rules_df = pd.DataFrame({
    'Rules': rules,
    'Left': list(map(lambda x: x.lhs, rules)),    
    'Right': list(map(lambda x: x.rhs, rules)),
    'Support': list(map(lambda x: x.support, rules)),
    'Confidence': list(map(lambda x: x.confidence, rules)),
})
rules_df

,Rules,Left,Right,Support,Confidence
0,{US} -> {Cabernet Sauvignon},"(US,)","(Cabernet Sauvignon,)",0.056289,0.134229
1,{Cabernet Sauvignon} -> {US},"(Cabernet Sauvignon,)","(US,)",0.056289,0.771730
2,{Pinot Noir} -> {California},"(Pinot Noir,)","(California,)",0.053081,0.518644
3,{California} -> {Pinot Noir},"(California,)","(Pinot Noir,)",0.053081,0.190333
4,{US} -> {California},"(US,)","(California,)",0.278885,0.665034
5,{California} -> {US},"(California,)","(US,)",0.278885,1.000000
6,{Virginie Boone} -> {California},"(Virginie Boone,)","(California,)",0.073339,0.999476
7,{California} -> {Virginie Boone},"(California,)","(Virginie Boone,)",0.073339,0.262973
8,{US} -> {Chardonnay},"(US,)","(Chardonnay,)",0.052350,0.124835
9,{Chardonnay} -> {US},"(Chardonnay,)","(US,)",0.052350,0.578768


### 4. 评价关联规则

- Lift指标

In [9]:
def lift(rule):
    observed_support = rule.count_full / rule.num_transactions
    prod_counts = rule.count_lhs * rule.count_rhs
    expected_support = (prod_counts) / rule.num_transactions ** 2
    return observed_support / expected_support

In [10]:
rules_df['Lift'] = list(map(lambda x: lift(x), rules))
rules_df

,Rules,Left,Right,Support,Confidence,Lift
0,{US} -> {Cabernet Sauvignon},"(US,)","(Cabernet Sauvignon,)",0.056289,0.134229,1.840278
1,{Cabernet Sauvignon} -> {US},"(Cabernet Sauvignon,)","(US,)",0.056289,0.771730,1.840278
2,{Pinot Noir} -> {California},"(Pinot Noir,)","(California,)",0.053081,0.518644,1.859703
3,{California} -> {Pinot Noir},"(California,)","(Pinot Noir,)",0.053081,0.190333,1.859703
4,{US} -> {California},"(US,)","(California,)",0.278885,0.665034,2.384614
5,{California} -> {US},"(California,)","(US,)",0.278885,1.000000,2.384614
6,{Virginie Boone} -> {California},"(Virginie Boone,)","(California,)",0.073339,0.999476,3.583824
7,{California} -> {Virginie Boone},"(California,)","(Virginie Boone,)",0.073339,0.262973,3.583824
8,{US} -> {Chardonnay},"(US,)","(Chardonnay,)",0.052350,0.124835,1.380139
9,{Chardonnay} -> {US},"(Chardonnay,)","(US,)",0.052350,0.578768,1.380139


- Conviction指标

In [11]:
def conviction(self):
    eps = 10e-10  # Avoid zero division
    prob_not_rhs = 1 - self.count_rhs / self.num_transactions
    prob_not_rhs_given_lhs = 1 - self.confidence
    return prob_not_rhs / (prob_not_rhs_given_lhs + eps)

In [12]:
rules_df['Conviction'] = list(map(lambda x: conviction(x), rules))
rules_df

,Rules,Left,Right,Support,Confidence,Lift,Conviction
0,{US} -> {Cabernet Sauvignon},"(US,)","(Cabernet Sauvignon,)",0.056289,0.134229,1.840278,1.070792e+00
1,{Cabernet Sauvignon} -> {US},"(Cabernet Sauvignon,)","(US,)",0.056289,0.771730,1.840278,2.543675e+00
2,{Pinot Noir} -> {California},"(Pinot Noir,)","(California,)",0.053081,0.518644,1.859703,1.498090e+00
3,{California} -> {Pinot Noir},"(California,)","(Pinot Noir,)",0.053081,0.190333,1.859703,1.108671e+00
4,{US} -> {California},"(US,)","(California,)",0.278885,0.665034,2.384614,2.152798e+00
5,{California} -> {US},"(California,)","(US,)",0.278885,1.000000,2.384614,5.806449e+08
6,{Virginie Boone} -> {California},"(Virginie Boone,)","(California,)",0.073339,0.999476,3.583824,1.375452e+03
7,{California} -> {Virginie Boone},"(California,)","(Virginie Boone,)",0.073339,0.262973,3.583824,1.257244e+00
8,{US} -> {Chardonnay},"(US,)","(Chardonnay,)",0.052350,0.124835,1.380139,1.039289e+00
9,{Chardonnay} -> {US},"(Chardonnay,)","(US,)",0.052350,0.578768,1.380139,1.378445e+00


### 5. 可视化展示并对挖掘结果进行分析

- 可视化展示

挖掘的结果展示如下：

In [13]:
rules_df

,Rules,Left,Right,Support,Confidence,Lift,Conviction
0,{US} -> {Cabernet Sauvignon},"(US,)","(Cabernet Sauvignon,)",0.056289,0.134229,1.840278,1.070792e+00
1,{Cabernet Sauvignon} -> {US},"(Cabernet Sauvignon,)","(US,)",0.056289,0.771730,1.840278,2.543675e+00
2,{Pinot Noir} -> {California},"(Pinot Noir,)","(California,)",0.053081,0.518644,1.859703,1.498090e+00
3,{California} -> {Pinot Noir},"(California,)","(Pinot Noir,)",0.053081,0.190333,1.859703,1.108671e+00
4,{US} -> {California},"(US,)","(California,)",0.278885,0.665034,2.384614,2.152798e+00
5,{California} -> {US},"(California,)","(US,)",0.278885,1.000000,2.384614,5.806449e+08
6,{Virginie Boone} -> {California},"(Virginie Boone,)","(California,)",0.073339,0.999476,3.583824,1.375452e+03
7,{California} -> {Virginie Boone},"(California,)","(Virginie Boone,)",0.073339,0.262973,3.583824,1.257244e+00
8,{US} -> {Chardonnay},"(US,)","(Chardonnay,)",0.052350,0.124835,1.380139,1.039289e+00
9,{Chardonnay} -> {US},"(Chardonnay,)","(US,)",0.052350,0.578768,1.380139,1.378445e+00


- 对挖掘结果进行分析

   根据上述挖掘结果，我们可以挖掘出一些关于**葡萄酒种类与原产地**之间、以及**品酒师与酒类原产地**之间的有趣关系,。  
例如：  
Cabernet Sauvignon(卡百内红葡萄酒)很有可能来源于US(美国)。  
Pinot Noir（黑皮诺葡萄酒）很有可能来源于California（加利福尼亚州）。  
Chardonnay（夏敦埃酒）很有可能来源于US（美国）。  
Virginie Boone（人名）经常品鉴California（加利福尼亚州）产的葡萄酒。
等一系列的属性及其相关值之间的有趣信息。